https://qiita.com/umi_mori/items/757834e0ef75f38cea19


In [1]:
! pip install pyserial

In [6]:
!  ls -l /dev/tty.*

crw-rw-rw-  1 root  wheel  0x900000a Feb  5 19:05 /dev/tty.Bluetooth-Incoming-Port
crw-rw-rw-  1 root  wheel  0x9000008 Feb  5 19:05 /dev/tty.ESP32test2
crw-rw-rw-  1 root  wheel  0x9000002 Feb  5 19:05 /dev/tty.JBLXtreme3
crw-rw-rw-  1 root  wheel  0x9000006 Feb  5 19:05 /dev/tty.NorisPowerbeats
crw-rw-rw-  1 root  wheel  0x9000004 Feb  5 19:05 /dev/tty.NoriyukiFujimurasSoloPro
crw-rw-rw-  1 root  wheel  0x900000c Feb  5 19:10 /dev/tty.usbserial-54BB0211361
crw-rw-rw-  1 root  wheel  0x900000e Feb  5 19:10 /dev/tty.wchusbserial54BB0211361
crw-rw-rw-  1 root  wheel  0x9000000 Feb  5 19:04 /dev/tty.wlan-debug


## ASCII
https://qiita.com/umi_mori/items/757834e0ef75f38cea19

In [9]:
import serial

serialCommand = "test text\n"
writeSer = serial.Serial('/dev/tty.wchusbserial54BB0211361',115200, timeout=3)
writeSer.write(serialCommand.encode())
writeSer.close()

## Binary
https://qiita.com/Gyutan/items/150c9fac57f7380a1763
LEDは1m６０個。　５m＝３００個　４色で１２００bytes

配列の初期化　https://yu-nix.com/archives/python-array-init/

In [54]:
import serial

length = 101
data = [0] * length 
print(data)
data[8] = 254
data[(length-1)] = 255
print(data)
# convert list to bytearray
send_binary =bytearray(data) 

# pyserialで送信
with serial.Serial('/dev/tty.wchusbserial54BB0211361', 500000) as ser:
    print('---')
    ser.write(send_binary)
    print('---')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 254, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255]
---
---


## 　画面インタラクションで送る

cv2 マウスインタラクション https://rasp.hateblo.jp/entry/2016/01/24/204539

cv2 ウインドウ　https://dev.classmethod.jp/articles/open-and-close-an-image-in-a-window-in-opencv/

cv２ウインドウ　気をつけること　https://dev.classmethod.jp/articles/importing-and-displaying-images-in-python-opencv/

cv2 画像なしでウインドウを作る？　https://qiita.com/tokkuri/items/ab6eb4f97e08b4334e07


In [ ]:
import cv2
import numpy as np

def mouse_event(event, x, y, flags, param):
    #print("mosuse event")
    cv2.rectangle(canvas, (0,100), (300,200), (0, 0, 0), cv2.FILLED, cv2.LINE_AA)
    cv2.rectangle(canvas, (x,100), (x+1,200), (127, 127, 127), cv2.FILLED, cv2.LINE_AA)
    #cv2.circle(canvas, (x, y), 50, (0, 255, 255), -1)
    cv2.imshow("X", canvas)

canvas = np.zeros((200,300,3), np.uint8)
cv2.putText(canvas, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow("X", canvas)
cv2.setMouseCallback("X", mouse_event)
key = cv2.waitKey(0)

if key == ord('q'):            #qを押した時の処理
    cv2.waitKey(1)
    cv2.destroyAllWindows()  
    cv2.waitKey(1)

# notebookでのcv２のウインドウ閉じる方法
https://qiita.com/kemako/items/fd72c65ca964a1b74fef

# Serialでマウス移動を送る

In [16]:
from timeit import default_timer as timer
import cv2

class DispFps():
    def __init__(self):
        # 表示関連定義
        self.__width = 80
        self.__height = 20
        self.__font_size = 0.4
        self.__font_width = 1
        self.__font_style = cv2.FONT_HERSHEY_COMPLEX
        self.__font_color = (255, 255, 255)
        self.__background_color = (0, 0, 0)

        # フレーム数カウント用変数
        self.__frame_count = 0

        # FPS計算用変数
        self.__accum_time = 0
        self.__curr_fps = 0
        self.__prev_time = timer()
        self.__str = "FPS: "
    
    def __calc(self):
        # フレーム数更新
        self.__frame_count += 1

        # FPS更新
        self.__curr_time = timer()
        self.__exec_time = self.__curr_time - self.__prev_time
        self.__prev_time = self.__curr_time
        self.__accum_time = self.__accum_time + self.__exec_time
        self.__curr_fps = self.__curr_fps + 1
        if self.__accum_time > 1:
            self.__accum_time = self.__accum_time - 1
            self.__str = "FPS: " + str(self.__curr_fps)
            self.__curr_fps = 0

    def __disp(self, frame, str, x1, y1, x2, y2):
        cv2.rectangle(frame, (x1, y1), (x2, y2), self.__background_color, -1)
        cv2.putText(frame, str, (x1 + 5, y2 - 5), self.__font_style, self.__font_size, self.__font_color, self.__font_width)

    def disp(self, frame):
        # 表示内容計算
        self.__calc()
        # フレーム数（左上に表示する）
        self.__disp(frame, str(self.__frame_count), 0, 0, x2 = self.__width, y2 = self.__height)
        # FPS(右上に表示する)
        screen_width = int(frame.shape[1])
        self.__disp(frame, self.__str, screen_width - self.__width, 0, screen_width, self.__height)

In [17]:
import cv2
import numpy as np
import serial
import time

length = 300

'''
#ser = serial.Serial('/dev/tty.wchusbserial54BB0211361', 115200)
ser = serial.Serial()
ser.braudrate = 115200
ser.port = '/dev/tty.wchusbserial54BB0211361'
ser.open()
'''


def mouse_event(event, x, y, flags, param):
    print("mouse event")
    cv2.rectangle(canvas, (0,100), (300,200), (0, 0, 0), cv2.FILLED, cv2.LINE_AA)
    cv2.rectangle(canvas, (x,100), (x+1,200), (127, 127, 127), cv2.FILLED, cv2.LINE_AA)
    #cv2.circle(canvas, (x, y), 50, (0, 255, 255), -1)
    dispFps.disp(canvas)
    cv2.imshow("X", canvas)
    
    #last array is for stop char
    data = [0] * (length+1) 
    if x<length:
        data[x] = 254
    
    data[length] = 255
    send_binary =bytes(data)
    #print(data)
    
    ser.write(send_binary)
    
    time.sleep(0.05)
    
    '''
    try:
        if ser.isOpen():
            ser.write(send_binary)
            print('write')
        else:
            ser = serial.Serial('/dev/tty.wchusbserial54BB0211361', 115200)
        time.sleep(0.05)
    except NameError:
        print('not found')
        time.sleep(0.05)
        '''

ser = serial.Serial('/dev/tty.wchusbserial54BB0211361', 115200)
print(ser.portstr)
dispFps = DispFps()
    
#last array is for stop char
data = [0] * (length+1) 
print(data)
data[length] = 255
print(data)
# convert list to bytearray


canvas = np.zeros((200,300,3), np.uint8)
cv2.putText(canvas, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow("X", canvas)
cv2.setMouseCallback("X", mouse_event)
key = cv2.waitKey(0)

if key == ord('q'):            #qを押した時の処理
    cv2.waitKey(1)
    cv2.destroyAllWindows()  
    ser.close()
    cv2.waitKey(1)

/dev/tty.wchusbserial54BB0211361
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Camera's fps setting
https://www.klv.co.jp/corner/python-opencv-camera-framerate.html